In [1]:
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import csv
from pandas.plotting import scatter_matrix
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVC
from sklearn import ensemble
from sklearn.metrics import log_loss
plt.style.use('ggplot')
%matplotlib inline
pd.options.display.width = 0
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime

In [2]:
chunk = pd.read_csv('train.tar.gz', compression='gzip', header=0, sep=';', quotechar='"', nrows = 1000000)

try:
    del chunk['CG3']
    del chunk['CG2']
    del chunk['CG1']
    del chunk['C7']
    del chunk['l2']
    del chunk['train.csv']
    del chunk['C11']
except Exception:
    pass

categorical_columns = ['label']
numerical_columns = ['C1', 'C2', 'C4', 'C10', 'l1','C5', 'C12','C9','C8','C6','C3']
chunk = chunk.fillna(chunk.median(axis=0), axis=0)
for c in categorical_columns:
    chunk[c] = chunk[c].astype('category')
    data_describe = chunk[c].describe()
    chunk[c] = chunk[c].fillna(data_describe['top'])
binary_columns    = ['label']
# nonbinary_columns = ['C7', 'l2']

data_numerical = chunk[numerical_columns]
# data_nonbinary = pd.get_dummies(chunk[nonbinary_columns])
data_numerical = (data_numerical - data_numerical.mean()) / data_numerical.std()
chunk = pd.concat((data_numerical, chunk[binary_columns]), axis=1)
chunk = pd.DataFrame(chunk, dtype=float)





X = chunk.drop(('label'), axis=1)
y = chunk['label']
feature_names = X.columns



KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=5, p=2,
          weights='uniform')


In [12]:

knn = ensemble.GradientBoostingRegressor(n_estimators=400)
# knn = KNeighborsRegressor()
knn.fit(X, y)
print(knn)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=400, n_iter_no_change=None, presort='auto',
             random_state=None, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)


In [3]:
y_train_predict = knn.predict(X)


err_train = log_loss(y, y_train_predict)
# err_test  = np.mean(y_test  != y_test_predict)
print(err_train)

0.00667543553534043


In [4]:

importances = knn.feature_importances_
indices = np.argsort(importances)[::-1]

print("Feature importances:")
for f, idx in enumerate(indices):
    print("{:2d}. feature '{:5s}' ({:.4f})".format(f + 1, feature_names[idx], importances[idx]))
    


AttributeError: 'KNeighborsRegressor' object has no attribute 'feature_importances_'

In [5]:
test_chunk = pd.read_csv('test-data.tar.gz', compression='gzip', header=0, sep=',', quotechar='"')


In [6]:
try:
    del test_chunk['CG3']
    del test_chunk['CG2']
    del test_chunk['CG1']
    del test_chunk['C7']
    del test_chunk['l2']
    del test_chunk['test.csv']
    del chunk['C11']
except Exception:
    pass

categorical_columns = ['label']
numerical_columns = ['C1', 'C2', 'C4', 'C10', 'l1', 'C5', 'C12','C9','C8','C6','C3']
test_chunk = test_chunk.fillna(test_chunk.median(axis=0), axis=0)
for c in categorical_columns:
    test_chunk[c] = test_chunk[c].astype('category')
    data_describe = test_chunk[c].describe()
    test_chunk[c] = test_chunk[c].fillna(data_describe['top'])
binary_columns    = ['label']
# nonbinary_columns = ['C7', 'l2']

data_numerical = test_chunk[numerical_columns]
# data_nonbinary = pd.get_dummies(chunk[nonbinary_columns])
data_numerical = (data_numerical - data_numerical.mean()) / data_numerical.std()
test_chunk = pd.concat((data_numerical, test_chunk[binary_columns]), axis=1)
test_chunk = pd.DataFrame(test_chunk, dtype=float)

X_test = test_chunk.drop(('label'), axis=1)
y_test = test_chunk['label']
feature_names = X_test.columns

In [13]:
y_test_predict = knn.predict(X_test)
# err_test  = np.mean(y_test  != y_test_predict)


In [14]:
with open('answer.csv','w') as file:
    for line in y_test_predict:
        file.write(str(line))
        file.write('\n')